<a href="https://colab.research.google.com/github/tahammarkalkan/deneme/blob/main/kanser_%C3%B6nerimi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Veri setini yükle (Ensure 'lung_cancer_data.csv' is in the same directory or provide the full path)
# Veri setinizin dosya adı 'lung_cancer_data.csv' ve ayırıcı 'noktalı virgül' (;) olarak varsayılmıştır.
# Added encoding='latin-1' to handle potential non-UTF-8 characters
try:
    df = pd.read_csv('lung_cancer_data.csv', delimiter=';', encoding='latin-1')
except UnicodeDecodeError:
    # If 'latin-1' fails, try another common encoding like 'ISO-8859-1' or 'cp1252'
    print("Latin-1 encoding failed, trying ISO-8859-1...")
    try:
        df = pd.read_csv('lung_cancer_data.csv', delimiter=';', encoding='ISO-8859-1')
    except UnicodeDecodeError:
        print("ISO-8859-1 encoding failed, trying cp1252...")
        try:
            df = pd.read_csv('lung_cancer_data.csv', delimiter=';', encoding='cp1252')
        except Exception as e:
            print(f"Could not read the file with common encodings. Error: {e}")
            # Handle the error or exit as appropriate for your application
            exit() # Or raise the exception
except FileNotFoundError:
    print("Error: 'lung_cancer_data.csv' not found. Please ensure the file is in the correct directory.")
    exit() # Or raise the exception


# 'Person Cigarette Smoking History Pack Year Value' sütununu sayısal değere dönüştür.
# Dönüşümde hata oluşursa (örn. metin değerleri), bu hücreleri NaN (Not a Number) olarak işaretle.
df['Person Cigarette Smoking History Pack Year Value'] = pd.to_numeric(
    df['Person Cigarette Smoking History Pack Year Value'], errors='coerce'
)

# Hedef değişken olan 'Overall Survival Status' sütununda NaN değerler içeren satırları temizle.
# Bu, modelin hedef değişkensiz veri üzerinde eğitilmesini engeller.
df_cleaned = df.dropna(subset=['Overall Survival Status']).copy()

# Tahmin etmek istediğimiz hedef değişkeni tanımla.
target_variable = 'Overall Survival Status'

# Tahmin süreci için işlevsiz veya az katkı sağlayan sütunları tanımla.
# Bu sütunlar genellikle:
# - Kimlik bilgileri (Patient ID, Sample ID vb.)
# - Çok yüksek oranda eksik veriye sahip sütunlar
# - Doğrudan model için kullanılamayan serbest metin veya tarih sütunları
# - Tanıdan ziyade tedaviye veya sonuca daha çok odaklanan bilgiler
# - Diğer özelliklerle yüksek korelasyon gösteren veya gereksiz tekrar içerenler
columns_to_exclude = [
    'Patient ID', 'Sample ID', 'Other Patient ID', 'Other Sample ID', 'Pathology report uuid',
    'ALK Analysis Type', 'American Joint Committee on Cancer Publication Version Type',
    'Disease code', 'Kras mutation identified type', 'Longest Dimension',
    'Shortest Dimension', 'Specimen Second Longest Dimension', 'Sample Initial Weight',
    'TMB (nonsynonymous)', 'Vial number', 'Days to Sample Collection.',
    'Form completion date', 'Pathology Report File Name', 'ICD-10 Classification',
    'International Classification of Diseases for Oncology, Third Edition ICD-O-3 Histology Code',
    'International Classification of Diseases for Oncology, Third Edition ICD-O-3 Site Code',
    'Oncotree Code', 'Tumor Site', 'Location lung parenchyma', 'Primary Other Site of Disease Name',
    'Cancer Study', 'Cancer Type', 'Cancer Type Detailed', 'Neoplasm Histologic Type Name',
    'Adjuvant Postoperative Targeted Therapy Administered Indicator',
    'Did patient start adjuvant postoperative radiotherapy?',
    'Neoadjuvant Therapy Type Administered Prior To Resection Text',
    'Primary Therapy Outcome Success Type', 'New Neoplasm Event Post Initial Therapy Indicator',
    'Surgical Margin Resection Status',
    'Number of Samples Per Patient', 'Informed consent verified', 'Is FFPE',
    'Karnofsky Performance Score', 'Performance Status', 'Performance Status Assessment Timepoint Category',
    'Prior Cancer Diagnosis Occurence', 'Project code', 'Pulmonary function test indicator',
    'Somatic Status', 'Started Smoking Year', 'Stopped Smoking Year', 'Year Cancer Initial Diagnosis',
    'Mutation Status', 'Mutation Type',
    'KRAS Mutation',
    'Neoplasm Disease Lymph Node Stage American Joint Committee on Cancer Code',
    'Neoplasm Disease Stage American Joint Committee on Cancer Code',
    'Fev1 fvc ratio postbroncholiator', 'Fev1 fvc ratio prebroncholiator',
    'Fev1 percent ref postbroncholiator', 'Fev1 percent ref prebroncholiator',
]

# Özellik setini (X) ve hedef değişkeni (y) oluştur.
# Dışlanacak sütunları ve hedef değişkeni orijinal DataFrame'den çıkar.
X = df_cleaned.drop(columns=columns_to_exclude + [target_variable], errors='ignore', axis=1)
y = df_cleaned[target_variable]

# Sayısal ve kategorik özellikleri tanımla.
# Bu listeler, yukarıdaki dışlama işleminden sonra kalan ve model için önemli görülen sütunlardır.
numerical_features = [
    'Diagnosis Age',
    'Mutation Count',
    'Fraction Genome Altered',
    'Carbon monoxide diffusion dlco',
    'Person Cigarette Smoking History Pack Year Value'
]
categorical_features = [
    'Sex',
    'Ethnicity Category',
    'Race Category',
    'Sample Type',
    'Patient Smoking History Category',
    'American Joint Committee on Cancer Metastasis Stage Code',
    'American Joint Committee on Cancer Tumor Stage Code',
    'ALK Translocation Status'
]

# Özellik listelerinin, X DataFrame'inde gerçekten mevcut olduğundan emin ol.
# Bu, herhangi bir sütun adı hatası veya önceki veri temizleme adımlarından kaynaklanan sorunları önler.
numerical_features = [f for f in numerical_features if f in X.columns]
categorical_features = [f for f in categorical_features if f in X.columns]

# Sayısal özellikler için bir ön işleme pipeline'ı oluştur:
# 1. Eksik değerleri medyan ile doldur (SimpleImputer).
# 2. Verileri standart ölçeklendirme (StandardScaler) ile normalize et.
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Kategorik özellikler için bir ön işleme pipeline'ı oluştur:
# 1. Eksik değerleri en sık görülen değer (mod) ile doldur (SimpleImputer).
# 2. Kategorik verileri sayısal formatlara dönüştürmek için One-Hot Encoding yap (OneHotEncoder).
#    'handle_unknown='ignore'' yeni veya bilinmeyen kategorileri görmezden gelmesini sağlar.
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Tüm ön işleme adımlarını birleştiren bir ColumnTransformer oluştur.
# Bu, farklı sütun türlerine farklı ön işleme adımları uygulamasını sağlar.
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features), # Sayısal sütunlara sayısal transformer uygula
        ('cat', categorical_transformer, categorical_features) # Kategorik sütunlara kategorik transformer uygula
    ])

# --- Makine Öğrenimi Modelleri ---

# Random Forest Sınıflandırıcısı için bir pipeline oluştur:
# 1. Veri ön işleme (yukarıdaki preprocessor).
# 2. Random Forest sınıflandırıcısını uygula.
rf_model = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', RandomForestClassifier(random_state=42))])

# Lojistik Regresyon için bir pipeline oluştur:
# 1. Veri ön işleme (yukarıdaki preprocessor).
# 2. Lojistik Regresyon sınıflandırıcısını uygula.
#    'solver='liblinear'' küçük veri setleri için iyi çalışan bir optimizasyon algoritmasıdır.
lr_model = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', LogisticRegression(random_state=42, solver='liblinear'))])

# Veriyi eğitim ve test setlerine ayır.
# Verilerin %80'i eğitim için, %20'si test için kullanılacak.
# 'random_state' sabitleme, kodun her çalıştırıldığında aynı bölünmeyi sağlar.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Modelleri Eğit ve Değerlendir ---

# Random Forest Modelini Eğit ve Sonuçları Yazdır
print("Random Forest Sınıflandırıcısı Sonuçları:")
rf_model.fit(X_train, y_train) # Modeli eğitim verileri üzerinde eğit
rf_predictions = rf_model.predict(X_test) # Test verileri üzerinde tahmin yap
print(f"Doğruluk (Accuracy): {accuracy_score(y_test, rf_predictions):.2f}") # Doğruluk skorunu hesapla
print("Sınıflandırma Raporu:\n", classification_report(y_test, rf_predictions)) # Detaylı sınıflandırma raporu

# Lojistik Regresyon Modelini Eğit ve Sonuçları Yazdır
print("\nLojistik Regresyon Sonuçları:")
lr_model.fit(X_train, y_train) # Modeli eğitim verileri üzerinde eğit
lr_predictions = lr_model.predict(X_test) # Test verileri üzerinde tahmin yap
print(f"Doğruluk (Accuracy): {accuracy_score(y_test, lr_predictions):.2f}") # Doğruluk skorunu hesapla
print("Sınıflandırma Raporu:\n", classification_report(y_test, lr_predictions)) # Detaylı sınıflandırma raporu

print("\n--- Kişiselleştirilmiş Tedavi Önerisi Yaklaşımı ---")
print("Bu sistem, aşağıdaki yollarla kişiselleştirilmiş tedavi önerileri için genişletilebilir:")
print("1. Hastanın tedavi geçmişi ve sonuçları gibi verileri (örn. tümör yanıtı, ilaç yan etkileri) modele dahil etmek.")
print("2. Farmakogenomik veriler (örn. ALK, EGFR, KRAS gibi spesifik gen mutasyonları) kullanarak ilaç duyarlılığını tahmin etmek.")
print("3. Biyosensör tabanlı mikroakışkan çiplerden elde edilen gerçek zamanlı verileri (örn. ilaca hücresel yanıt) model için ek özellikler olarak kullanmak.")
print("4. Belirli ilaç rejimlerini önermek veya etkinliklerini tahmin etmek için çok etiketli sınıflandırma veya regresyon modellerinden yararlanmak.")

Random Forest Sınıflandırıcısı Sonuçları:
Doğruluk (Accuracy): 0.66
Sınıflandırma Raporu:
               precision    recall  f1-score   support

    0:LIVING       0.77      0.80      0.78        75
  1:DECEASED       0.25      0.22      0.23        23

    accuracy                           0.66        98
   macro avg       0.51      0.51      0.51        98
weighted avg       0.65      0.66      0.65        98


Lojistik Regresyon Sonuçları:
Doğruluk (Accuracy): 0.65
Sınıflandırma Raporu:
               precision    recall  f1-score   support

    0:LIVING       0.78      0.76      0.77        75
  1:DECEASED       0.28      0.30      0.29        23

    accuracy                           0.65        98
   macro avg       0.53      0.53      0.53        98
weighted avg       0.66      0.65      0.66        98


--- Kişiselleştirilmiş Tedavi Önerisi Yaklaşımı ---
Bu sistem, aşağıdaki yollarla kişiselleştirilmiş tedavi önerileri için genişletilebilir:
1. Hastanın tedavi geçmişi ve son

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Veri setini yükle (Ensure 'lung_cancer_data.csv' is in the same directory or provide the full path)
# Veri setinizin dosya adı 'lung_cancer_data.csv' ve ayırıcı 'noktalı virgül' (;) olarak varsayılmıştır.
# Added encoding='latin-1' to handle potential non-UTF-8 characters
try:
    df = pd.read_csv('lung_cancer_data.csv', delimiter=';', encoding='latin-1')
except UnicodeDecodeError:
    # If 'latin-1' fails, try another common encoding like 'ISO-8859-1' or 'cp1252'
    print("Latin-1 encoding failed, trying ISO-8859-1...")
    try:
        df = pd.read_csv('lung_cancer_data.csv', delimiter=';', encoding='ISO-8859-1')
    except UnicodeDecodeError:
        print("ISO-8859-1 encoding failed, trying cp1252...")
        try:
            df = pd.read_csv('lung_cancer_data.csv', delimiter=';', encoding='cp1252')
        except Exception as e:
            print(f"Could not read the file with common encodings. Error: {e}")
            # Handle the error or exit as appropriate for your application
            exit() # Or raise the exception
except FileNotFoundError:
    print("Error: 'lung_cancer_data.csv' not found. Please ensure the file is in the correct directory.")
    exit() # Or raise the exception


# 'Person Cigarette Smoking History Pack Year Value' sütununu sayısal değere dönüştür.
# Dönüşümde hata oluşursa (örn. metin değerleri), bu hücreleri NaN (Not a Number) olarak işaretle.
df['Person Cigarette Smoking History Pack Year Value'] = pd.to_numeric(
    df['Person Cigarette Smoking History Pack Year Value'], errors='coerce'
)

# Hedef değişken olan 'Overall Survival Status' sütununda NaN değerler içeren satırları temizle.
# Bu, modelin hedef değiskensiz veri üzerinde eğitilmesini engeller.
df_cleaned = df.dropna(subset=['Overall Survival Status']).copy()

# Yeni hedef değişkeni: Kanserin evresi ('Neoplasm Disease Stage American Joint Committee on Cancer Code')
# Bu sütunda da NaN değerler olabilir, bu yüzden bu sütundaki NaN'ları da temizliyoruz.
df_cleaned = df_cleaned.dropna(subset=['Neoplasm Disease Stage American Joint Committee on Cancer Code']).copy()


# Tahmin etmek istediğimiz hedef değişkenleri tanımla.
target_diagnosis = 'Overall Survival Status'
target_stage = 'Neoplasm Disease Stage American Joint Committee on Cancer Code' # Kanserin evresi

# Tahmin süreci için işlevsiz veya az katkı sağlayan sütunları tanımla.
# Bu sütunlar genellikle:
# - Kimlik bilgileri (Patient ID, Sample ID vb.)
# - Çok yüksek oranda eksik veriye sahip sütunlar
# - Doğrudan model için kullanılamayan serbest metin veya tarih sütunları
# - Tanıdan ziyade tedaviye veya sonuca daha çok odaklanan bilgiler
# - Diğer özelliklerle yüksek korelasyon gösteren veya gereksiz tekrar içerenler
columns_to_exclude = [
    'Patient ID', 'Sample ID', 'Other Patient ID', 'Other Sample ID', 'Pathology report uuid',
    'ALK Analysis Type', 'American Joint Committee on Cancer Publication Version Type',
    'Disease code', 'Kras mutation identified type', 'Longest Dimension',
    'Shortest Dimension', 'Specimen Second Longest Dimension', 'Sample Initial Weight',
    'TMB (nonsynonymous)', 'Vial number', 'Days to Sample Collection.',
    'Form completion date', 'Pathology Report File Name', 'ICD-10 Classification',
    'International Classification of Diseases for Oncology, Third Edition ICD-O-3 Histology Code',
    'International Classification of Diseases for Oncology, Third Edition ICD-O-3 Site Code',
    'Oncotree Code', 'Tumor Site', 'Location lung parenchyma', 'Primary Other Site of Disease Name',
    'Cancer Study', 'Cancer Type', 'Cancer Type Detailed', 'Neoplasm Histologic Type Name',
    'Adjuvant Postoperative Targeted Therapy Administered Indicator',
    'Did patient start adjuvant postoperative radiotherapy?',
    'Neoadjuvant Therapy Type Administered Prior To Resection Text',
    'Primary Therapy Outcome Success Type', 'New Neoplasm Event Post Initial Therapy Indicator',
    'Surgical Margin Resection Status',
    'Number of Samples Per Patient', 'Informed consent verified', 'Is FFPE',
    'Karnofsky Performance Score', 'Performance Status', 'Performance Status Assessment Timepoint Category',
    'Prior Cancer Diagnosis Occurence', 'Project code', 'Pulmonary function test indicator',
    'Somatic Status', 'Started Smoking Year', 'Stopped Smoking Year', 'Year Cancer Initial Diagnosis',
    'Mutation Status', 'Mutation Type',
    'KRAS Mutation',
    'Neoplasm Disease Lymph Node Stage American Joint Committee on Cancer Code', # Kanser Evresi ile ilgili ancak ana evre sütunu daha genel
    'Fev1 fvc ratio postbroncholiator', 'Fev1 fvc ratio prebroncholiator',
    'Fev1 percent ref postbroncholiator', 'Fev1 percent ref prebroncholiator',
]

# Özellik setini (X) ve hedef değişkenleri (y_diagnosis, y_stage) oluştur.
# Dışlanacak sütunları ve hedef değişkenleri orijinal DataFrame'den çıkar.
# Her iki hedef değişken de çıkarılmalıdır ki X sadece özellikleri içersin.
X = df_cleaned.drop(columns=columns_to_exclude + [target_diagnosis, target_stage], errors='ignore', axis=1)
y_diagnosis = df_cleaned[target_diagnosis]
y_stage = df_cleaned[target_stage]

# Sayısal ve kategorik özellikleri tanımla.
# Bu listeler, yukarıdaki dışlama işleminden sonra kalan ve model için önemli görülen sütunlardır.
numerical_features = [
    'Diagnosis Age',
    'Mutation Count',
    'Fraction Genome Altered',
    'Carbon monoxide diffusion dlco',
    'Person Cigarette Smoking History Pack Year Value'
]
categorical_features = [
    'Sex',
    'Ethnicity Category',
    'Race Category',
    'Sample Type',
    'Patient Smoking History Category',
    'American Joint Committee on Cancer Metastasis Stage Code',
    'American Joint Committee on Cancer Tumor Stage Code',
    'ALK Translocation Status'
]

# Özellik listelerinin, X DataFrame'inde gerçekten mevcut olduğundan emin ol.
# Bu, herhangi bir sütun adı hatası veya önceki veri temizleme adımlarından kaynaklanan sorunları önler.
numerical_features = [f for f in numerical_features if f in X.columns]
categorical_features = [f for f in categorical_features if f in X.columns]

# Sayısal özellikler için bir ön işleme pipeline'ı oluştur:
# 1. Eksik değerleri medyan ile doldur (SimpleImputer).
# 2. Verileri standart ölçeklendirme (StandardScaler) ile normalize et.
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Kategorik özellikler için bir ön işleme pipeline'ı oluştur:
# 1. Eksik değerleri en sık görülen değer (mod) ile doldur (SimpleImputer).
# 2. Kategorik verileri sayısal formatlara dönüştürmek için One-Hot Encoding yap (OneHotEncoder).
#    'handle_unknown='ignore'' yeni veya bilinmeyen kategorileri görmezden gelmesini sağlar.
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Tüm ön işleme adımlarını birleştiren bir ColumnTransformer oluştur.
# Bu, farklı sütun türlerine farklı ön işleme adımları uygulamasını sağlar.
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features), # Sayısal sütunlara sayısal transformer uygula
        ('cat', categorical_transformer, categorical_features) # Kategorik sütunlara kategorik transformer uygula
    ])

# --- Makine Öğrenimi Modelleri ---

# 1. Kansere Yakalanma Durumu (Teşhis) için Model
# Random Forest Sınıflandırıcısı pipeline'ı
rf_diagnosis_model = Pipeline(steps=[('preprocessor', preprocessor),
                                     ('classifier', RandomForestClassifier(random_state=42))])

# Lojistik Regresyon pipeline'ı
lr_diagnosis_model = Pipeline(steps=[('preprocessor', preprocessor),
                                     ('classifier', LogisticRegression(random_state=42, solver='liblinear'))])

# 2. Kanser Evresi için Model
# Random Forest Sınıflandırıcısı pipeline'ı (Evre için)
rf_stage_model = Pipeline(steps=[('preprocessor', preprocessor),
                                 ('classifier', RandomForestClassifier(random_state=42))])

# Lojistik Regresyon pipeline'ı (Evre için)
lr_stage_model = Pipeline(steps=[('preprocessor', preprocessor),
                                 ('classifier', LogisticRegression(random_state=42, solver='liblinear'))])

# Veriyi eğitim ve test setlerine ayır.
# Bu X ve y_diagnosis için kullanılacak.
X_train_diag, X_test_diag, y_train_diag, y_test_diag = train_test_split(X, y_diagnosis, test_size=0.2, random_state=42)

# Evre tahmini için de aynı şekilde veri ayırma yapabiliriz.
# X ve y_stage için kullanılacak.
X_train_stage, X_test_stage, y_train_stage, y_test_stage = train_test_split(X, y_stage, test_size=0.2, random_state=42)


# --- Modelleri Eğit ve Değerlendir ---

print("--- Kansere Yakalanma Durumu (Teşhis) Modeli Sonuçları ---")
# Random Forest Modelini Eğit ve Sonuçları Yazdır (Teşhis)
print("\nRandom Forest Sınıflandırıcısı (Teşhis) Sonuçları:")
rf_diagnosis_model.fit(X_train_diag, y_train_diag)
rf_diagnosis_predictions = rf_diagnosis_model.predict(X_test_diag)
print(f"Doğruluk (Accuracy): {accuracy_score(y_test_diag, rf_diagnosis_predictions):.2f}")
print("Sınıflandırma Raporu:\n", classification_report(y_test_diag, rf_diagnosis_predictions))

# Lojistik Regresyon Modelini Eğit ve Sonuçları Yazdır (Teşhis)
print("\nLojistik Regresyon (Teşhis) Sonuçları:")
lr_diagnosis_model.fit(X_train_diag, y_train_diag)
lr_diagnosis_predictions = lr_diagnosis_model.predict(X_test_diag)
print(f"Doğruluk (Accuracy): {accuracy_score(y_test_diag, lr_diagnosis_predictions):.2f}")
print("Sınıflandırma Raporu:\n", classification_report(y_test_diag, lr_diagnosis_predictions))


print("\n--- Kanser Evresi Modeli Sonuçları ---")
# Random Forest Modelini Eğit ve Sonuçları Yazdır (Evre)
print("\nRandom Forest Sınıflandırıcısı (Evre) Sonuçları:")
rf_stage_model.fit(X_train_stage, y_train_stage)
rf_stage_predictions = rf_stage_model.predict(X_test_stage)
print(f"Doğruluk (Accuracy): {accuracy_score(y_test_stage, rf_stage_predictions):.2f}")
print("Sınıflandırma Raporu:\n", classification_report(y_test_stage, rf_stage_predictions))

# Lojistik Regresyon Modelini Eğit ve Sonuçları Yazdır (Evre)
print("\nLojistik Regresyon (Evre) Sonuçları:")
lr_stage_model.fit(X_train_stage, y_train_stage)
lr_stage_predictions = lr_stage_model.predict(X_test_stage)
print(f"Doğruluk (Accuracy): {accuracy_score(y_test_stage, lr_stage_predictions):.2f}")
print("Sınıflandırma Raporu:\n", classification_report(y_test_stage, lr_stage_predictions))


# --- Yeni Hasta Verileri ile Teşhis ve Öneri Fonksiyonu ---

def predict_and_recommend(patient_data: pd.DataFrame, diagnosis_model: Pipeline, stage_model: Pipeline):
    """
    Yeni hasta verilerini kullanarak kanser teşhisi, evresi ve tedavi önerisi yapar.

    Args:
        patient_data (pd.DataFrame): Tek bir hastanın özelliklerini içeren DataFrame.
                                     Sütun adları eğitimde kullanılanlarla aynı olmalıdır.
        diagnosis_model (Pipeline): Kansere yakalanma durumu (teşhis) için eğitilmiş pipeline modeli.
        stage_model (Pipeline): Kanser evresi için eğitilmiş pipeline modeli.

    Returns:
        dict: Teşhis, evre ve tedavi önerisini içeren bir sözlük.
    """
    # Hastanın teşhis durumunu tahmin et
    predicted_diagnosis = diagnosis_model.predict(patient_data)[0]
    predicted_diagnosis_proba = diagnosis_model.predict_proba(patient_data)[0]

    # Hastanın kanser evresini tahmin et
    predicted_stage = stage_model.predict(patient_data)[0]
    predicted_stage_proba = stage_model.predict_proba(patient_data)[0]

    # Sınıf olasılıklarını anlaşılır bir formatta göster
    diagnosis_labels = diagnosis_model.classes_
    diagnosis_prob_dict = {label: f"{prob:.2f}" for label, prob in zip(diagnosis_labels, predicted_diagnosis_proba)}

    stage_labels = stage_model.classes_
    stage_prob_dict = {label: f"{prob:.2f}" for label, prob in zip(stage_labels, predicted_stage_proba)}


    # Tedavi önerisi (veri setinde spesifik ilaç/tedavi bilgisi olmadığı için konseptsel)
    treatment_recommendation = ""
    if predicted_diagnosis == '0:LIVING': # Hastalığın olmadığı veya kontrol altında olduğu varsayımı
        treatment_recommendation = "Kanser teşhis edilmedi veya kontrol altında. Düzenli kontroller ve sağlıklı yaşam tarzı önerilir."
    elif predicted_diagnosis == '1:DECEASED': # Hastalığın ilerlediği varsayımı
        treatment_recommendation = "Kanser teşhis edildi. Durum kritik, acil tıbbi müdahale gereklidir."
    else: # Genel kanser teşhisi durumunda
        if "Stage I" in predicted_stage or "Stage IA" in predicted_stage or "Stage IB" in predicted_stage:
            treatment_recommendation = (
                "Evre I akciğer kanseri teşhisi kondu. Genel öneriler: Cerrahi rezeksiyon (tümörün çıkarılması) birincil tedavi yöntemidir. "
                "Bazı durumlarda adjuvan kemoterapi veya radyoterapi düşünülebilir. Biyosensör çipi ile tümör duyarlılığı analizi yapılabilir."
            )
        elif "Stage II" in predicted_stage or "Stage IIA" in predicted_stage or "Stage IIB" in predicted_stage:
            treatment_recommendation = (
                "Evre II akciğer kanseri teşhisi kondu. Genel öneriler: Cerrahi rezeksiyon genellikle uygulanır. "
                "Ameliyat sonrası adjuvan kemoterapi ve/veya radyoterapi sıklıkla önerilir. Hedefe yönelik tedaviler ve immünoterapi potansiyel seçeneklerdir. "
                "Mikroakışkan çip ile ilaç duyarlılığı testleri faydalı olabilir."
            )
        elif "Stage III" in predicted_stage or "Stage IIIA" in predicted_stage or "Stage IIIB" in predicted_stage or "Stage IIIC" in predicted_stage:
            treatment_recommendation = (
                "Evre III akciğer kanseri teşhisi kondu. Genel öneriler: Kombine multimodal tedavi (kemoterapi ve radyoterapi) yaygın olarak kullanılır. "
                "Bazı durumlarda cerrahi düşünülebilir. Hedefe yönelik tedaviler ve immünoterapi önemli rol oynar. "
                "Kişiselleştirilmiş tedavi için genetik analiz ve biyosensör tabanlı testler kritik önem taşır."
            )
        elif "Stage IV" in predicted_stage:
            treatment_recommendation = (
                "Evre IV akciğer kanseri teşhisi kondu (metastatik). Genel öneriler: Sistemik tedavi (kemoterapi, hedefe yönelik tedaviler, immünoterapi) birincil yaklaşımdır. "
                "Semptom yönetimi ve yaşam kalitesi ön plandadır. Bireysel genetik profile uygun en etkili ilaç kombinasyonlarını belirlemek için biyosensör ve farmakogenetik analizler çok değerlidir."
            )
        else:
            treatment_recommendation = (
                "Kanser teşhisi kondu ancak evresi belirsiz veya farklı bir evreleme sınıflandırmasına göre. "
                "Detaylı inceleme ve kişiselleştirilmiş tedavi planı için Onkoloji uzmanına danışılmalıdır. "
                "Farmakogenetik testler ve biyosensör tabanlı ilaç duyarlılık analizleri, en uygun tedavi seçeneğini belirlemede yardımcı olabilir."
            )

    return {
        "Teşhis Durumu": predicted_diagnosis,
        "Teşhis Olasılıkları": diagnosis_prob_dict,
        "Kanser Evresi": predicted_stage,
        "Kanser Evresi Olasılıkları": stage_prob_dict,
        "Tedavi Önerisi": treatment_recommendation
    }

# --- Örnek Yeni Hasta Verileri Oluşturma ---
# Bu kısım, yeni bir hastanın verilerini simüle etmek için kullanılır.
# Gerçek uygulamada, bu veriler bir formdan, elektronik sağlık kaydından vb. gelecektir.
# Sütun adlarının ve veri tiplerinin eğitimde kullanılanlarla eşleştiğinden emin olun.
# 'X' DataFrame'inizin sütunlarını referans alınız.

# Ortalama değerler veya örnek bir hasta profili
sample_patient_data = {
    'Diagnosis Age': 65,
    'Mutation Count': 100.0,
    'Fraction Genome Altered': 0.15,
    'Carbon monoxide diffusion dlco': 70.0,
    'Person Cigarette Smoking History Pack Year Value': 30.0,
    'Sex': 'Male',
    'Ethnicity Category': 'NOT HISPANIC OR LATINO',
    'Race Category': 'WHITE',
    'Sample Type': 'Primary',
    'Patient Smoking History Category': '3', # Eski sigara içen
    'American Joint Committee on Cancer Metastasis Stage Code': 'M0',
    'American Joint Committee on Cancer Tumor Stage Code': 'T2',
    'ALK Translocation Status': 'NO'
}

# Örnek hasta verilerini DataFrame'e dönüştür.
# Tek bir satır olduğu için index=[0] kullanıyoruz.
new_patient_df = pd.DataFrame([sample_patient_data])

# --- Yeni Hasta Verileri ile Tahmin ve Öneri Yapma ---
print("\n--- Yeni Hasta Verileri ile Teşhis ve Tedavi Önerisi ---")
# Random Forest modellerini kullanarak tahmin yapalım
rf_results = predict_and_recommend(new_patient_df, rf_diagnosis_model, rf_stage_model)
print("\nRandom Forest Modelleri ile Sonuçlar:")
for key, value in rf_results.items():
    print(f"{key}: {value}")

# Lojistik Regresyon modellerini kullanarak tahmin yapalım
lr_results = predict_and_recommend(new_patient_df, lr_diagnosis_model, lr_stage_model)
print("\nLojistik Regresyon Modelleri ile Sonuçlar:")
for key, value in lr_results.items():
    print(f"{key}: {value}")

--- Kansere Yakalanma Durumu (Teşhis) Modeli Sonuçları ---

Random Forest Sınıflandırıcısı (Teşhis) Sonuçları:
Doğruluk (Accuracy): 0.69
Sınıflandırma Raporu:
               precision    recall  f1-score   support

    0:LIVING       0.72      0.86      0.79        64
  1:DECEASED       0.57      0.36      0.44        33

    accuracy                           0.69        97
   macro avg       0.65      0.61      0.62        97
weighted avg       0.67      0.69      0.67        97


Lojistik Regresyon (Teşhis) Sonuçları:
Doğruluk (Accuracy): 0.65
Sınıflandırma Raporu:
               precision    recall  f1-score   support

    0:LIVING       0.71      0.78      0.75        64
  1:DECEASED       0.48      0.39      0.43        33

    accuracy                           0.65        97
   macro avg       0.60      0.59      0.59        97
weighted avg       0.64      0.65      0.64        97


--- Kanser Evresi Modeli Sonuçları ---

Random Forest Sınıflandırıcısı (Evre) Sonuçları:
Doğrulu

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import warnings

# Scikit-learn uyarılarını göz ardı et (çıktıyı daha temiz tutmak için)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Veri setini yükle (Ensure 'lung_cancer_data.csv' is in the same directory or provide the full path)
# Veri setinizin dosya adı 'lung_cancer_data.csv' ve ayırıcı 'noktalı virgül' (;) olarak varsayılmıştır.
df = pd.read_csv('lung_cancer_data.csv', delimiter=';')

# 'Person Cigarette Smoking History Pack Year Value' sütununu sayısal değere dönüştür.
# Dönüşümde hata oluşursa (örn. metin değerleri), bu hücreleri NaN (Not a Number) olarak işaretle.
df['Person Cigarette Smoking History Pack Year Value'] = pd.to_numeric(
    df['Person Cigarette Smoking History Pack Year Value'], errors='coerce'
)

# Hedef değişken olan 'Overall Survival Status' sütununda NaN değerler içeren satırları temizle.
# Bu, modelin hedef değişkensiz veri üzerinde eğitilmesini engeller.
df_cleaned = df.dropna(subset=['Overall Survival Status']).copy()

# Yeni hedef değişkeni: Kanserin evresi ('Neoplasm Disease Stage American Joint Committee on Cancer Code')
# Bu sütunda da NaN değerler olabilir, bu yüzden bu sütundaki NaN'ları da temizliyoruz.
df_cleaned = df_cleaned.dropna(subset=['Neoplasm Disease Stage American Joint Committee on Cancer Code']).copy()


# Tahmin etmek istediğimiz hedef değişkenleri tanımla.
target_diagnosis = 'Overall Survival Status'
target_stage = 'Neoplasm Disease Stage American Joint Committee on Cancer Code' # Kanserin evresi

# Tahmin süreci için işlevsiz veya az katkı sağlayan sütunları tanımla.
# Bu sütunlar genellikle:
# - Kimlik bilgileri (Patient ID, Sample ID vb.)
# - Çok yüksek oranda eksik veriye sahip sütunlar
# - Doğrudan model için kullanılamayan serbest metin veya tarih sütunları
# - Tanıdan ziyade tedaviye veya sonuca daha çok odaklanan bilgiler
# - Diğer özelliklerle yüksek korelasyon gösteren veya gereksiz tekrar içerenler
columns_to_exclude = [
    'Patient ID', 'Sample ID', 'Other Patient ID', 'Other Sample ID', 'Pathology report uuid',
    'ALK Analysis Type', 'American Joint Committee on Cancer Publication Version Type',
    'Disease code', 'Kras mutation identified type', 'Longest Dimension',
    'Shortest Dimension', 'Specimen Second Longest Dimension', 'Sample Initial Weight',
    'TMB (nonsynonymous)', 'Vial number', 'Days to Sample Collection.',
    'Form completion date', 'Pathology Report File Name', 'ICD-10 Classification',
    'International Classification of Diseases for Oncology, Third Edition ICD-O-3 Histology Code',
    'International Classification of Diseases for Oncology, Third Edition ICD-O-3 Site Code',
    'Oncotree Code', 'Tumor Site', 'Location lung parenchyma', 'Primary Other Site of Disease Name',
    'Cancer Study', 'Cancer Type', 'Cancer Type Detailed', 'Neoplasm Histologic Type Name',
    'Adjuvant Postoperative Targeted Therapy Administered Indicator',
    'Did patient start adjuvant postoperative radiotherapy?',
    'Neoadjuvant Therapy Type Administered Prior To Resection Text',
    'Primary Therapy Outcome Success Type', 'New Neoplasm Event Post Initial Therapy Indicator',
    'Surgical Margin Resection Status',
    'Number of Samples Per Patient', 'Informed consent verified', 'Is FFPE',
    'Karnofsky Performance Score', 'Performance Status', 'Performance Status Assessment Timepoint Category',
    'Prior Cancer Diagnosis Occurence', 'Project code', 'Pulmonary function test indicator',
    'Somatic Status', 'Started Smoking Year', 'Stopped Smoking Year', 'Year Cancer Initial Diagnosis',
    'Mutation Status', 'Mutation Type',
    'KRAS Mutation',
    'Neoplasm Disease Lymph Node Stage American Joint Committee on Cancer Code',
    'Fev1 fvc ratio postbroncholiator', 'Fev1 fvc ratio prebroncholiator',
    'Fev1 percent ref postbroncholiator', 'Fev1 percent ref prebroncholiator',
]

# Özellik setini (X) ve hedef değişkenleri (y_diagnosis, y_stage) oluştur.
# Dışlanacak sütunları ve hedef değişkenleri orijinal DataFrame'den çıkar.
# Her iki hedef değişken de çıkarılmalıdır ki X sadece özellikleri içersin.
X = df_cleaned.drop(columns=columns_to_exclude + [target_diagnosis, target_stage], errors='ignore', axis=1)
y_diagnosis = df_cleaned[target_diagnosis]
y_stage = df_cleaned[target_stage]

# Sayısal ve kategorik özellikleri tanımla.
# Bu listeler, yukarıdaki dışlama işleminden sonra kalan ve model için önemli görülen sütunlardır.
numerical_features = [
    'Diagnosis Age',
    'Mutation Count',
    'Fraction Genome Altered',
    'Carbon monoxide diffusion dlco',
    'Person Cigarette Smoking History Pack Year Value'
]
categorical_features = [
    'Sex',
    'Ethnicity Category',
    'Race Category',
    'Sample Type',
    'Patient Smoking History Category',
    'American Joint Committee on Cancer Metastasis Stage Code',
    'American Joint Committee on Cancer Tumor Stage Code',
    'ALK Translocation Status'
]

# Özellik listelerinin, X DataFrame'inde gerçekten mevcut olduğundan emin ol.
# Bu, herhangi bir sütun adı hatası veya önceki veri temizleme adımlarından kaynaklanan sorunları önler.
numerical_features = [f for f in numerical_features if f in X.columns]
categorical_features = [f for f in categorical_features if f in X.columns]

# Sayısal özellikler için bir ön işleme pipeline'ı oluştur:
# 1. Eksik değerleri medyan ile doldur (SimpleImputer).
# 2. Verileri standart ölçeklendirme (StandardScaler) ile normalize et.
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Kategorik özellikler için bir ön işleme pipeline'ı oluştur:
# 1. Eksik değerleri en sık görülen değer (mod) ile doldur (SimpleImputer).
# 2. Kategorik verileri sayısal formatlara dönüştürmek için One-Hot Encoding yap (OneHotEncoder).
#    'handle_unknown='ignore'' yeni veya bilinmeyen kategorileri görmezden gelmesini sağlar.
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Tüm ön işleme adımlarını birleştiren bir ColumnTransformer oluştur.
# Bu, farklı sütun türlerine farklı ön işleme adımları uygulamasını sağlar.
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features), # Sayısal sütunlara sayısal transformer uygula
        ('cat', categorical_transformer, categorical_features) # Kategorik sütunlara kategorik transformer uygula
    ])

# --- Makine Öğrenimi Modelleri ---

# 1. Kansere Yakalanma Durumu (Teşhis) için Model
# Random Forest Sınıflandırıcısı pipeline'ı
rf_diagnosis_model = Pipeline(steps=[('preprocessor', preprocessor),
                                     ('classifier', RandomForestClassifier(random_state=42))])

# Lojistik Regresyon pipeline'ı
lr_diagnosis_model = Pipeline(steps=[('preprocessor', preprocessor),
                                     ('classifier', LogisticRegression(random_state=42, solver='liblinear'))])

# 2. Kanser Evresi için Model
# Random Forest Sınıflandırıcısı pipeline'ı (Evre için)
rf_stage_model = Pipeline(steps=[('preprocessor', preprocessor),
                                 ('classifier', RandomForestClassifier(random_state=42))])

# Lojistik Regresyon pipeline'ı (Evre için)
lr_stage_model = Pipeline(steps=[('preprocessor', preprocessor),
                                 ('classifier', LogisticRegression(random_state=42, solver='liblinear'))])

# Veriyi eğitim ve test setlerine ayır.
# Bu X ve y_diagnosis için kullanılacak.
X_train_diag, X_test_diag, y_train_diag, y_test_diag = train_test_split(X, y_diagnosis, test_size=0.2, random_state=42)

# Evre tahmini için de aynı şekilde veri ayırma yapabiliriz.
# X ve y_stage için kullanılacak.
X_train_stage, X_test_stage, y_train_stage, y_test_stage = train_test_split(X, y_stage, test_size=0.2, random_state=42)


# --- Modelleri Eğit ve Değerlendir ---

print("--- Kansere Yakalanma Durumu (Teşhis) Modeli Sonuçları ---")
# Random Forest Modelini Eğit ve Sonuçları Yazdır (Teşhis)
print("\nRandom Forest Sınıflandırıcısı (Teşhis) Eğitiliyor ve Değerlendiriliyor...")
rf_diagnosis_model.fit(X_train_diag, y_train_diag)
rf_diagnosis_predictions = rf_diagnosis_model.predict(X_test_diag)
print(f"Doğruluk (Accuracy): {accuracy_score(y_test_diag, rf_diagnosis_predictions):.2f}")
print("Sınıflandırma Raporu:\n", classification_report(y_test_diag, rf_diagnosis_predictions))

# Lojistik Regresyon Modelini Eğit ve Sonuçları Yazdır (Teşhis)
print("\nLojistik Regresyon (Teşhis) Eğitiliyor ve Değerlendiriliyor...")
lr_diagnosis_model.fit(X_train_diag, y_train_diag)
lr_diagnosis_predictions = lr_diagnosis_model.predict(X_test_diag)
print(f"Doğruluk (Accuracy): {accuracy_score(y_test_diag, lr_diagnosis_predictions):.2f}")
print("Sınıflandırma Raporu:\n", classification_report(y_test_diag, lr_diagnosis_predictions))


print("\n--- Kanser Evresi Modeli Sonuçları ---")
# Random Forest Modelini Eğit ve Sonuçları Yazdır (Evre)
print("\nRandom Forest Sınıflandırıcısı (Evre) Eğitiliyor ve Değerlendiriliyor...")
rf_stage_model.fit(X_train_stage, y_train_stage)
rf_stage_predictions = rf_stage_model.predict(X_test_stage)
print(f"Doğruluk (Accuracy): {accuracy_score(y_test_stage, rf_stage_predictions):.2f}")
print("Sınıflandırma Raporu:\n", classification_report(y_test_stage, rf_stage_predictions))

# Lojistik Regresyon Modelini Eğit ve Sonuçları Yazdır (Evre)
print("\nLojistik Regresyon (Evre) Eğitiliyor ve Değerlendiriliyor...")
lr_stage_model.fit(X_train_stage, y_train_stage)
lr_stage_predictions = lr_stage_model.predict(X_test_stage)
print(f"Doğruluk (Accuracy): {accuracy_score(y_test_stage, lr_stage_predictions):.2f}")
print("Sınıflandırma Raporu:\n", classification_report(y_test_stage, lr_stage_predictions))


# --- Yeni Hasta Verileri ile Teşhis ve Öneri Fonksiyonu ---

def predict_and_recommend(patient_data: pd.DataFrame, diagnosis_model: Pipeline, stage_model: Pipeline, feature_columns: list):
    """
    Yeni hasta verilerini kullanarak kanser teşhisi, evresi ve tedavi önerisi yapar.

    Args:
        patient_data (pd.DataFrame): Tek bir hastanın özelliklerini içeren DataFrame.
                                     Sütun adları eğitimde kullanılanlarla aynı olmalıdır.
        diagnosis_model (Pipeline): Kansere yakalanma durumu (teşhis) için eğitilmiş pipeline modeli.
        stage_model (Pipeline): Kanser evresi için eğitilmiş pipeline modeli.
        feature_columns (list): Modelin beklediği tüm özellik sütunlarının listesi.

    Returns:
        dict: Teşhis, evre ve tedavi önerisini içeren bir sözlük.
    """
    # Hastanın DataFrame'inin modelin beklediği tüm sütunlara sahip olduğundan emin olun.
    # Eksik sütunları NaN ile doldur, gereksizleri çıkar.
    patient_data_processed = pd.DataFrame(columns=feature_columns)
    for col in feature_columns:
        if col in patient_data.columns:
            patient_data_processed[col] = patient_data[col]
        else:
            patient_data_processed[col] = pd.NA # Veya uygun bir varsayılan değer

    # Hastanın teşhis durumunu tahmin et
    predicted_diagnosis = diagnosis_model.predict(patient_data_processed)[0]
    predicted_diagnosis_proba = diagnosis_model.predict_proba(patient_data_processed)[0]

    # Hastanın kanser evresini tahmin et
    predicted_stage = stage_model.predict(patient_data_processed)[0]
    predicted_stage_proba = stage_model.predict_proba(patient_data_processed)[0]

    # Sınıf olasılıklarını anlaşılır bir formatta göster
    diagnosis_labels = diagnosis_model.classes_
    diagnosis_prob_dict = {label: f"{prob:.2f}" for label, prob in zip(diagnosis_labels, predicted_diagnosis_proba)}

    stage_labels = stage_model.classes_
    stage_prob_dict = {label: f"{prob:.2f}" for label, prob in zip(stage_labels, predicted_stage_proba)}


    # Tedavi önerisi (veri setinde spesifik ilaç/tedavi bilgisi olmadığı için konseptsel)
    # Bu bölüm, tahmin edilen evreye göre genel tıbbi öneriler sunar.
    treatment_recommendation = ""
    if predicted_diagnosis == '0:LIVING':
        treatment_recommendation = "Kanser teşhis edilmedi veya iyi seyrediyor. Düzenli kontroller ve sağlıklı yaşam tarzı önerilir."
    elif predicted_diagnosis == '1:DECEASED':
        treatment_recommendation = "Kanser teşhis edildi ve maalesef hastalık ilerlemiş görünüyor. Acil ve kapsamlı tıbbi değerlendirme gereklidir."
    else: # Genel kanser teşhisi durumunda (buraya düşmesi beklenmez, ancak güvenlik için)
        treatment_recommendation = "Kanser teşhisi kondu. Detaylı değerlendirme için bir onkolog ile görüşünüz."

    # Kanser evresine göre tedavi önerisini özelleştir
    # Bu öneriler genel tıbbi bilgiler olup, kişisel durumlar için bir doktora danışılmalıdır.
    if predicted_diagnosis != '0:LIVING': # Sadece kanser teşhis edildiyse evreye göre öneri yap
        if "Stage I" in predicted_stage or "Stage IA" in predicted_stage or "Stage IB" in predicted_stage:
            treatment_recommendation += (
                "\nTedavi Önerisi (Evre I): Cerrahi rezeksiyon (tümörün çıkarılması) birincil tedavi yöntemidir. "
                "Bazı durumlarda adjuvan (tamamlayıcı) kemoterapi veya radyoterapi düşünülebilir. "
                "Farmakogenetik testler ve biyosensör çipleri, tümörün ilaca duyarlılığını analiz etmek için kullanılabilir."
            )
        elif "Stage II" in predicted_stage or "Stage IIA" in predicted_stage or "Stage IIB" in predicted_stage:
            treatment_recommendation += (
                "\nTedavi Önerisi (Evre II): Cerrahi rezeksiyon genellikle uygulanır. "
                "Ameliyat sonrası adjuvan kemoterapi ve/veya radyoterapi sıklıkla önerilir. "
                "Hedefe yönelik tedaviler ve immünoterapi potansiyel seçeneklerdir. "
                "Mikroakışkan çip ile ilaç duyarlılığı testleri, en uygun tedavi protokolünü belirlemede faydalı olabilir."
            )
        elif "Stage III" in predicted_stage or "Stage IIIA" in predicted_stage or "Stage IIIB" in predicted_stage or "Stage IIIC" in predicted_stage:
            treatment_recommendation += (
                "\nTedavi Önerisi (Evre III): Kombine multimodal tedavi (kemoterapi ve radyoterapi) yaygın olarak kullanılır. "
                "Bazı durumlarda cerrahi düşünülebilir. Hedefe yönelik tedaviler ve immünoterapi önemli rol oynar. "
                "Kişiselleştirilmiş tedavi için genetik analiz ve biyosensör tabanlı testler kritik önem taşır, çünkü bu evrede tedavi karmaşıktır."
            )
        elif "Stage IV" in predicted_stage:
            treatment_recommendation += (
                "\nTedavi Önerisi (Evre IV - Metastatik): Sistemik tedavi (kemoterapi, hedefe yönelik tedaviler, immünoterapi) birincil yaklaşımdır. "
                "Semptom yönetimi ve yaşam kalitesi ön plandadır. Bireysel genetik profile uygun en etkili ilaç kombinasyonlarını belirlemek için biyosensör ve farmakogenetik analizler çok değerlidir."
            )
        else: # Evre bilinmiyorsa veya beklenmedik bir formatta ise
            treatment_recommendation += (
                "\nTedavi Önerisi: Kanser evresi belirsiz. Detaylı inceleme ve kişiselleştirilmiş tedavi planı için "
                "bir Onkoloji uzmanına danışılmalıdır. Farmakogenetik testler ve biyosensör tabanlı ilaç duyarlılık analizleri "
                "en uygun tedavi seçeneğini belirlemede yardımcı olabilir."
            )

    return {
        "Teşhis Durumu": predicted_diagnosis,
        "Teşhis Olasılıkları": diagnosis_prob_dict,
        "Kanser Evresi": predicted_stage,
        "Kanser Evresi Olasılıkları": stage_prob_dict,
        "Tedavi Önerisi": treatment_recommendation
    }

# --- Kullanıcıdan Hasta Verilerini Alma ---
print("\n--- Yeni Hasta Verilerini Girin ---")
print("Lütfen aşağıdaki bilgileri girerek hastanın durumunu değerlendirelim.")

patient_input_data = {}

# Sayısal özellikler için giriş
for feature in numerical_features:
    while True:
        try:
            value = input(f"Lütfen '{feature}' değerini girin (Sayısal bir değer olmalı): ")
            patient_input_data[feature] = float(value)
            break
        except ValueError:
            print("Geçersiz giriş. Lütfen sayısal bir değer girin.")

# Kategorik özellikler için giriş
for feature in categorical_features:
    # Olası benzersiz değerleri kullanıcıya göstermek için bir liste oluşturabilirsiniz.
    # Ancak bu, kodun daha uzun olmasına neden olabilir. Şimdilik sadece input alıyoruz.
    # `handle_unknown='ignore'` sayesinde bilinmeyen girişler sorun yaratmayacaktır.
    if feature == 'Sex':
        print("\nCinsiyet için geçerli değerler: 'Male', 'Female'")
    elif feature == 'Patient Smoking History Category':
        print("\nSigara Geçmişi Kategorisi için geçerli değerler (1-4 arası, 4 en yüksek): '1', '2', '3', '4'")
    elif feature == 'American Joint Committee on Cancer Metastasis Stage Code':
        print("\nAJCC Metastaz Evre Kodu için geçerli değerler: 'M0', 'M1a', 'M1b', 'MX' vb. (Veri setindeki çeşitliliğe bakın)")
    elif feature == 'American Joint Committee on Cancer Tumor Stage Code':
        print("\nAJCC Tümör Evre Kodu için geçerli değerler: 'T1', 'T2', 'T3', 'T4', 'TX' vb. (Veri setindeki çeşitliliğe bakın)")
    elif feature == 'ALK Translocation Status':
        print("\nALK Translokasyon Durumu için geçerli değerler: 'YES', 'NO'")
    elif feature == 'Ethnicity Category':
        print("\nEtnik Köken Kategorisi için geçerli değerler: 'HISPANIC OR LATINO', 'NOT HISPANIC OR LATINO'")
    elif feature == 'Race Category':
        print("\nIrk Kategorisi için geçerli değerler: 'WHITE', 'BLACK OR AFRICAN AMERICAN', 'ASIAN', 'AMERICAN INDIAN OR ALASKA NATIVE', 'UNKNOWN'")
    elif feature == 'Sample Type':
        print("\nÖrnek Tipi için geçerli değerler: 'Primary', 'Recurrent', 'Metastatic', 'Cell Line'")


    value = input(f"Lütfen '{feature}' değerini girin: ")
    patient_input_data[feature] = value

# Kullanıcıdan alınan verileri DataFrame'e dönüştür.
# Tek bir satır olduğu için index=[0] kullanıyoruz.
new_patient_df = pd.DataFrame([patient_input_data])

# --- Yeni Hasta Verileri ile Tahmin ve Öneri Yapma ---
print("\n--- Analiz Sonuçları ---")
# Random Forest modellerini kullanarak tahmin yapalım
rf_results = predict_and_recommend(new_patient_df, rf_diagnosis_model, rf_stage_model, X.columns.tolist())
print("\n--- Random Forest Modelleri ile Sonuçlar ---")
for key, value in rf_results.items():
    print(f"{key}: {value}")

# Lojistik Regresyon modellerini kullanarak tahmin yapalım
lr_results = predict_and_recommend(new_patient_df, lr_diagnosis_model, lr_stage_model, X.columns.tolist())
print("\n--- Lojistik Regresyon Modelleri ile Sonuçlar ---")
for key, value in lr_results.items():
    print(f"{key}: {value}")

print("\n--- Önemli Not ---")
print("Bu sistem, mevcut veri setindeki kalıpları kullanarak bir ön teşhis ve öneri sunar.")
print("Ancak, yapay zeka tarafından yapılan tüm tahminler **sadece bilgilendirme amaçlıdır** ve tıbbi tavsiye yerine geçmez.")
print("Herhangi bir tıbbi teşhis, tedavi veya karar için daima nitelikli bir sağlık profesyoneline danışmanız esastır.")
print("Bu modelin doğruluğu, kullanılan veri setinin kalitesi, boyutu ve özelliklerinin kapsamına bağlıdır.")


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xde in position 2556: invalid continuation byte

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import warnings

# Scikit-learn uyarılarını göz ardı et (çıktıyı daha temiz tutmak için)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Veri setini yükle (Ensure 'lung_cancer_data.csv' is in the same directory or provide the full path)
# Veri setinizin dosya adı 'lung_cancer_data.csv' ve ayırıcı 'noktalı virgül' (;) olarak varsayılmıştır.
# Added encoding='latin-1' to handle potential non-UTF-8 characters
# Reintroducing the encoding parameter based on previous successful attempts
try:
    df = pd.read_csv('lung_cancer_data.csv', delimiter=';', encoding='latin-1')
except UnicodeDecodeError:
    # If 'latin-1' fails, try another common encoding like 'ISO-8859-1' or 'cp1252'
    print("Latin-1 encoding failed, trying ISO-8859-1...")
    try:
        df = pd.read_csv('lung_cancer_data.csv', delimiter=';', encoding='ISO-8859-1')
    except UnicodeDecodeError:
        print("ISO-8859-1 encoding failed, trying cp1252...")
        try:
            df = pd.read_csv('lung_cancer_data.csv', delimiter=';', encoding='cp1252')
        except Exception as e:
            print(f"Could not read the file with common encodings. Error: {e}")
            # Handle the error or exit as appropriate for your application
            # Removing exit() to allow the notebook to continue and show the error if reading truly fails
            raise e # Re-raise the exception after printing

except FileNotFoundError:
    print("Error: 'lung_cancer_data.csv' not found. Please ensure the file is in the correct directory.")
    # Removing exit() to allow the notebook to continue and show the error if file is missing
    raise FileNotFoundError("'lung_cancer_data.csv' not found.") # Re-raise the exception

# 'Person Cigarette Smoking History Pack Year Value' sütununu sayısal değere dönüştür.
# Dönüşümde hata oluşursa (örn. metin değerleri), bu hücreleri NaN (Not a Number) olarak işaretle.
df['Person Cigarette Smoking History Pack Year Value'] = pd.to_numeric(
    df['Person Cigarette Smoking History Pack Year Value'], errors='coerce'
)

# Hedef değişken olan 'Overall Survival Status' sütununda NaN değerler içeren satırları temizle.
# Bu, modelin hedef değişkensiz veri üzerinde eğitilmesini engeller.
df_cleaned = df.dropna(subset=['Overall Survival Status']).copy()

# Yeni hedef değişkeni: Kanserin evresi ('Neoplasm Disease Stage American Joint Committee on Cancer Code')
# Bu sütunda da NaN değerler olabilir, bu yüzden bu sütundaki NaN'ları da temizliyoruz.
df_cleaned = df_cleaned.dropna(subset=['Neoplasm Disease Stage American Joint Committee on Cancer Code']).copy()


# Tahmin etmek istediğimiz hedef değişkenleri tanımla.
target_diagnosis = 'Overall Survival Status'
target_stage = 'Neoplasm Disease Stage American Joint Committee on Cancer Code' # Kanserin evresi

# Tahmin süreci için işlevsiz veya az katkı sağlayan sütunları tanımla.
# Bu sütunlar genellikle:
# - Kimlik bilgileri (Patient ID, Sample ID vb.)
# - Çok yüksek oranda eksik veriye sahip sütunlar
# - Doğrudan model için kullanılamayan serbest metin veya tarih sütunları
# - Tanıdan ziyade tedaviye veya sonuca daha çok odaklanan bilgiler
# - Diğer özelliklerle yüksek korelasyon gösteren veya gereksiz tekrar içerenler
columns_to_exclude = [
    'Patient ID', 'Sample ID', 'Other Patient ID', 'Other Sample ID', 'Pathology report uuid',
    'ALK Analysis Type', 'American Joint Committee on Cancer Publication Version Type',
    'Disease code', 'Kras mutation identified type', 'Longest Dimension',
    'Shortest Dimension', 'Specimen Second Longest Dimension', 'Sample Initial Weight',
    'TMB (nonsynonymous)', 'Vial number', 'Days to Sample Collection.',
    'Form completion date', 'Pathology Report File Name', 'ICD-10 Classification',
    'International Classification of Diseases for Oncology, Third Edition ICD-O-3 Histology Code',
    'International Classification of Diseases for Oncology, Third Edition ICD-O-3 Site Code',
    'Oncotree Code', 'Tumor Site', 'Location lung parenchyma', 'Primary Other Site of Disease Name',
    'Cancer Study', 'Cancer Type', 'Cancer Type Detailed', 'Neoplasm Histologic Type Name',
    'Adjuvant Postoperative Targeted Therapy Administered Indicator',
    'Did patient start adjuvant postoperative radiotherapy?',
    'Neoadjuvant Therapy Type Administered Prior To Resection Text',
    'Primary Therapy Outcome Success Type', 'New Neoplasm Event Post Initial Therapy Indicator',
    'Surgical Margin Resection Status',
    'Number of Samples Per Patient', 'Informed consent verified', 'Is FFPE',
    'Karnofsky Performance Score', 'Performance Status', 'Performance Status Assessment Timepoint Category',
    'Prior Cancer Diagnosis Occurence', 'Project code', 'Pulmonary function test indicator',
    'Somatic Status', 'Started Smoking Year', 'Stopped Smoking Year', 'Year Cancer Initial Diagnosis',
    'Mutation Status', 'Mutation Type',
    'KRAS Mutation',
    'Neoplasm Disease Lymph Node Stage American Joint Committee on Cancer Code',
    'Fev1 fvc ratio postbroncholiator', 'Fev1 fvc ratio prebroncholiator',
    'Fev1 percent ref postbroncholiator', 'Fev1 percent ref prebroncholiator',
]

# Özellik setini (X) ve hedef değişkenleri (y_diagnosis, y_stage) oluştur.
# Dışlanacak sütunları ve hedef değişkenleri orijinal DataFrame'den çıkar.
# Her iki hedef değişken de çıkarılmalıdır ki X sadece özellikleri içersin.
X = df_cleaned.drop(columns=columns_to_exclude + [target_diagnosis, target_stage], errors='ignore', axis=1)
y_diagnosis = df_cleaned[target_diagnosis]
y_stage = df_cleaned[target_stage]

# Sayısal ve kategorik özellikleri tanımla.
# Bu listeler, yukarıdaki dışlama işleminden sonra kalan ve model için önemli görülen sütunlardır.
numerical_features = [
    'Diagnosis Age',
    'Mutation Count',
    'Fraction Genome Altered',
    'Carbon monoxide diffusion dlco',
    'Person Cigarette Smoking History Pack Year Value'
]
categorical_features = [
    'Sex',
    'Ethnicity Category',
    'Race Category',
    'Sample Type',
    'Patient Smoking History Category',
    'American Joint Committee on Cancer Metastasis Stage Code',
    'American Joint Committee on Cancer Tumor Stage Code',
    'ALK Translocation Status'
]

# Özellik listelerinin, X DataFrame'inde gerçekten mevcut olduğundan emin ol.
# Bu, herhangi bir sütun adı hatası veya önceki veri temizleme adımlarından kaynaklanan sorunları önler.
numerical_features = [f for f in numerical_features if f in X.columns]
categorical_features = [f for f in categorical_features if f in X.columns]

# Sayısal özellikler için bir ön işleme pipeline'ı oluştur:
# 1. Eksik değerleri medyan ile doldur (SimpleImputer).
# 2. Verileri standart ölçeklendirme (StandardScaler) ile normalize et.
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Kategorik özellikler için bir ön işleme pipeline'ı oluştur:
# 1. Eksik değerleri en sık görülen değer (mod) ile doldur (SimpleImputer).
# 2. Kategorik verileri sayısal formatlara dönüştürmek için One-Hot Encoding yap (OneHotEncoder).
#    'handle_unknown='ignore'' yeni veya bilinmeyen kategorileri görmezden gelmesini sağlar.
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Tüm ön işleme adımlarını birleştiren bir ColumnTransformer oluştur.
# Bu, farklı sütun türlerine farklı ön işleme adımları uygulamasını sağlar.
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features), # Sayısal sütunlara sayısal transformer uygula
        ('cat', categorical_transformer, categorical_features) # Kategorik sütunlara kategorik transformer uygula
    ])

# --- Makine Öğrenimi Modelleri ---

# 1. Kansere Yakalanma Durumu (Teşhis) için Model
# Random Forest Sınıflandırıcısı pipeline'ı
rf_diagnosis_model = Pipeline(steps=[('preprocessor', preprocessor),
                                     ('classifier', RandomForestClassifier(random_state=42))])

# Lojistik Regresyon pipeline'ı
lr_diagnosis_model = Pipeline(steps=[('preprocessor', preprocessor),
                                     ('classifier', LogisticRegression(random_state=42, solver='liblinear'))])

# 2. Kanser Evresi için Model
# Random Forest Sınıflandırıcısı pipeline'ı (Evre için)
rf_stage_model = Pipeline(steps=[('preprocessor', preprocessor),
                                 ('classifier', RandomForestClassifier(random_state=42))])

# Lojistik Regresyon pipeline'ı (Evre için)
lr_stage_model = Pipeline(steps=[('preprocessor', preprocessor),
                                 ('classifier', LogisticRegression(random_state=42, solver='liblinear'))])

# Veriyi eğitim ve test setlerine ayır.
# Bu X ve y_diagnosis için kullanılacak.
X_train_diag, X_test_diag, y_train_diag, y_test_diag = train_test_split(X, y_diagnosis, test_size=0.2, random_state=42)

# Evre tahmini için de aynı şekilde veri ayırma yapabiliriz.
# X ve y_stage için kullanılacak.
X_train_stage, X_test_stage, y_train_stage, y_test_stage = train_test_split(X, y_stage, test_size=0.2, random_state=42)


# --- Modelleri Eğit ve Değerlendir ---

print("--- Kansere Yakalanma Durumu (Teşhis) Modeli Sonuçları ---")
# Random Forest Modelini Eğit ve Sonuçları Yazdır (Teşhis)
print("\nRandom Forest Sınıflandırıcısı (Teşhis) Eğitiliyor ve Değerlendiriliyor...")
rf_diagnosis_model.fit(X_train_diag, y_train_diag)
rf_diagnosis_predictions = rf_diagnosis_model.predict(X_test_diag)
print(f"Doğruluk (Accuracy): {accuracy_score(y_test_diag, rf_diagnosis_predictions):.2f}")
print("Sınıflandırma Raporu:\n", classification_report(y_test_diag, rf_diagnosis_predictions))

# Lojistik Regresyon Modelini Eğit ve Sonuçları Yazdır (Teşhis)
print("\nLojistik Regresyon (Teşhis) Eğitiliyor ve Değerlendiriliyor...")
lr_diagnosis_model.fit(X_train_diag, y_train_diag)
lr_diagnosis_predictions = lr_diagnosis_model.predict(X_test_diag)
print(f"Doğruluk (Accuracy): {accuracy_score(y_test_diag, lr_diagnosis_predictions):.2f}")
print("Sınıflandırma Raporu:\n", classification_report(y_test_diag, lr_diagnosis_predictions))


print("\n--- Kanser Evresi Modeli Sonuçları ---")
# Random Forest Modelini Eğit ve Sonuçları Yazdır (Evre)
print("\nRandom Forest Sınıflandırıcısı (Evre) Eğitiliyor ve Değerlendiriliyor...")
rf_stage_model.fit(X_train_stage, y_train_stage)
rf_stage_predictions = rf_stage_model.predict(X_test_stage)
print(f"Doğruluk (Accuracy): {accuracy_score(y_test_stage, rf_stage_predictions):.2f}")
print("Sınıflandırma Raporu:\n", classification_report(y_test_stage, rf_stage_predictions))

# Lojistik Regresyon Modelini Eğit ve Sonuçları Yazdır (Evre)
print("\nLojistik Regresyon (Evre) Eğitiliyor ve Değerlendiriliyor...")
lr_stage_model.fit(X_train_stage, y_train_stage)
lr_stage_predictions = lr_stage_model.predict(X_test_stage)
print(f"Doğruluk (Accuracy): {accuracy_score(y_test_stage, lr_stage_predictions):.2f}")
print("Sınıflandırma Raporu:\n", classification_report(y_test_stage, lr_stage_predictions))


# --- Yeni Hasta Verileri ile Teşhis ve Öneri Fonksiyonu ---

def predict_and_recommend(patient_data: pd.DataFrame, diagnosis_model: Pipeline, stage_model: Pipeline, feature_columns: list):
    """
    Yeni hasta verilerini kullanarak kanser teşhisi, evresi ve tedavi önerisi yapar.

    Args:
        patient_data (pd.DataFrame): Tek bir hastanın özelliklerini içeren DataFrame.
                                     Sütun adları eğitimde kullanılanlarla aynı olmalıdır.
        diagnosis_model (Pipeline): Kansere yakalanma durumu (teşhis) için eğitilmiş pipeline modeli.
        stage_model (Pipeline): Kanser evresi için eğitilmiş pipeline modeli.
        feature_columns (list): Modelin beklediği tüm özellik sütunlarının listesi.

    Returns:
        dict: Teşhis, evre ve tedavi önerisini içeren bir sözlük.
    """
    # Hastanın DataFrame'inin modelin beklediği tüm sütunlara sahip olduğundan emin olun.
    # Eksik sütunları NaN ile doldur, gereksizleri çıkar.
    patient_data_processed = pd.DataFrame(columns=feature_columns)
    for col in feature_columns:
        if col in patient_data.columns:
            patient_data_processed[col] = patient_data[col]
        else:
            # Use None instead of pd.NA for older pandas versions if necessary, or ensure pandas is recent
            patient_data_processed[col] = pd.NA # Veya uygun bir varsayılan değer (genellikle None veya NaN)

    # Hastanın teşhis durumunu tahmin et
    # predict ve predict_proba çağrılmadan önce preprocessor adımının doğru çalıştığından emin olmak için transform edilebilir.
    # Ancak pipeline içindeki preprocessor bunu otomatik yapacaktır.
    predicted_diagnosis = diagnosis_model.predict(patient_data_processed)[0]
    predicted_diagnosis_proba = diagnosis_model.predict_proba(patient_data_processed)[0]

    # Hastanın kanser evresini tahmin et
    predicted_stage = stage_model.predict(patient_data_processed)[0]
    predicted_stage_proba = stage_model.predict_proba(patient_data_processed)[0]

    # Sınıf olasılıklarını anlaşılır bir formatta göster
    diagnosis_labels = diagnosis_model.classes_
    diagnosis_prob_dict = {label: f"{prob:.2f}" for label, prob in zip(diagnosis_labels, predicted_diagnosis_proba)}

    stage_labels = stage_model.classes_
    stage_prob_dict = {label: f"{prob:.2f}" for label, prob in zip(stage_labels, predicted_stage_proba)}


    # Tedavi önerisi (veri setinde spesifik ilaç/tedavi bilgisi olmadığı için konseptsel)
    # Bu bölüm, tahmin edilen evreye göre genel tıbbi öneriler sunar.
    treatment_recommendation = ""
    if predicted_diagnosis == '0:LIVING':
        treatment_recommendation = "Kanser teşhis edilmedi veya iyi seyrediyor. Düzenli kontroller ve sağlıklı yaşam tarzı önerilir."
    elif predicted_diagnosis == '1:DECEASED':
        treatment_recommendation = "Kanser teşhis edildi ve maalesef hastalık ilerlemiş görünüyor. Acil ve kapsamlı tıbbi değerlendirme gereklidir."
    else: # Genel kanser teşhisi durumunda (buraya düşmesi beklenmez, ancak güvenlik için)
        treatment_recommendation = "Kanser teşhisi kondu. Detaylı değerlendirme için bir onkolog ile görüşünüz."

    # Kanser evresine göre tedavi önerisini özelleştir
    # Bu öneriler genel tıbbi bilgiler olup, kişisel durumlar için bir doktora danışılmalıdır.
    if predicted_diagnosis != '0:LIVING': # Sadece kanser teşhis edildiyse evreye göre öneri yap
        if "Stage I" in predicted_stage or "Stage IA" in predicted_stage or "Stage IB" in predicted_stage:
            treatment_recommendation += (
                "\nTedavi Önerisi (Evre I): Cerrahi rezeksiyon (tümörün çıkarılması) birincil tedavi yöntemidir. "
                "Bazı durumlarda adjuvan (tamamlayıcı) kemoterapi veya radyoterapi düşünülebilir. "
                "Farmakogenetik testler ve biyosensör çipleri, tümörün ilaca duyarlılığını analiz etmek için kullanılabilir."
            )
        elif "Stage II" in predicted_stage or "Stage IIA" in predicted_stage or "Stage IIB" in predicted_stage:
            treatment_recommendation += (
                "\nTedavi Önerisi (Evre II): Cerrahi rezeksiyon genellikle uygulanır. "
                "Ameliyat sonrası adjuvan kemoterapi ve/veya radyoterapi sıklıkla önerilir. "
                "Hedefe yönelik tedaviler ve immünoterapi potansiyel seçeneklerdir. "
                "Mikroakışkan çip ile ilaç duyarlılığı testleri, en uygun tedavi protokolünü belirlemede faydalı olabilir."
            )
        elif "Stage III" in predicted_stage or "Stage IIIA" in predicted_stage or "Stage IIIB" in predicted_stage or "Stage IIIC" in predicted_stage:
            treatment_recommendation += (
                "\nTedavi Önerisi (Evre III): Kombine multimodal tedavi (kemoterapi ve radyoterapi) yaygın olarak kullanılır. "
                "Bazı durumlarda cerrahi düşünülebilir. Hedefe yönelik tedaviler ve immünoterapi önemli rol oynar. "
                "Kişiselleştirilmiş tedavi için genetik analiz ve biyosensör tabanlı testler kritik önem taşır, çünkü bu evrede tedavi karmaşıktır."
            )
        elif "Stage IV" in predicted_stage:
            treatment_recommendation += (
                "\nTedavi Önerisi (Evre IV - Metastatik): Sistemik tedavi (kemoterapi, hedefe yönelik tedaviler, immünoterapi) birincil yaklaşımdır. "
                "Semptom yönetimi ve yaşam kalitesi ön plandadır. Bireysel genetik profile uygun en etkili ilaç kombinasyonlarını belirlemek için biyosensör ve farmakogenetik analizler çok değerlidir."
            )
        else: # Evre bilinmiyorsa veya beklenmedik bir formatta ise
            treatment_recommendation += (
                "\nTedavi Önerisi: Kanser evresi belirsiz. Detaylı inceleme ve kişiselleştirilmiş tedavi planı için "
                "bir Onkoloji uzmanına danışılmalıdır. Farmakogenetik testler ve biyosensör tabanlı ilaç duyarlılık analizleri "
                "en uygun tedavi seçeneğini belirlemede yardımcı olabilir."
            )

    return {
        "Teşhis Durumu": predicted_diagnosis,
        "Teşhis Olasılıkları": diagnosis_prob_dict,
        "Kanser Evresi": predicted_stage,
        "Kanser Evresi Olasılıkları": stage_prob_dict,
        "Tedavi Önerisi": treatment_recommendation
    }

# --- Kullanıcıdan Hasta Verilerini Alma ---
print("\n--- Yeni Hasta Verilerini Girin ---")
print("Lütfen aşağıdaki bilgileri girerek hastanın durumunu değerlendirelim.")

patient_input_data = {}

# Sayısal özellikler için giriş
for feature in numerical_features:
    while True:
        try:
            value = input(f"Lütfen '{feature}' değerini girin (Sayısal bir değer olmalı): ")
            patient_input_data[feature] = float(value)
            break
        except ValueError:
            print("Geçersiz giriş. Lütfen sayısal bir değer girin.")

# Kategorik özellikler için giriş
for feature in categorical_features:
    # Olası benzersiz değerleri kullanıcıya göstermek için bir liste oluşturabilirsiniz.
    # Ancak bu, kodun daha uzun olmasına neden olabilir. Şimdilik sadece input alıyoruz.
    # `handle_unknown='ignore'` sayesinde bilinmeyen girişler sorun yaratmayacaktır.
    # Eğitim verisindeki benzersiz değerleri göstermek faydalı olabilir:
    # unique_values = df_cleaned[feature].unique().tolist()
    # print(f"\n'{feature}' için geçerli değerler (veride bulunan): {unique_values}")

    if feature == 'Sex':
        print("\nCinsiyet için geçerli değerler: 'Male', 'Female'")
    elif feature == 'Patient Smoking History Category':
        print("\nSigara Geçmişi Kategorisi için geçerli değerler (1-4 arası, 4 en yüksek): '1', '2', '3', '4'")
    elif feature == 'American Joint Committee on Cancer Metastasis Stage Code':
        print("\nAJCC Metastaz Evre Kodu için geçerli değerler: 'M0', 'M1a', 'M1b', 'MX' vb. (Veri setindeki çeşitliliğe bakın)")
    elif feature == 'American Joint Committee on Cancer Tumor Stage Code':
        print("\nAJCC Tümör Evre Kodu için geçerli değerler: 'T1', 'T2', 'T3', 'T4', 'TX' vb. (Veri setindeki çeşitliliğe bakın)")
    elif feature == 'ALK Translocation Status':
        print("\nALK Translokasyon Durumu için geçerli değerler: 'YES', 'NO'")
    elif feature == 'Ethnicity Category':
        print("\nEtnik Köken Kategorisi için geçerli değerler: 'HISPANIC OR LATINO', 'NOT HISPANIC OR LATINO'")
    elif feature == 'Race Category':
        print("\nIrk Kategorisi için geçerli değerler: 'WHITE', 'BLACK OR AFRICAN AMERICAN', 'ASIAN', 'AMERICAN INDIAN OR ALASKA NATIVE', 'UNKNOWN'")
    elif feature == 'Sample Type':
        print("\nÖrnek Tipi için geçerli değerler: 'Primary', 'Recurrent', 'Metastatic', 'Cell Line'")


    value = input(f"Lütfen '{feature}' değerini girin: ")
    patient_input_data[feature] = value

# Kullanıcıdan alınan verileri DataFrame'e dönüştür.
# Tek bir satır olduğu için index=[0] kullanıyoruz.
new_patient_df = pd.DataFrame([patient_input_data])

# --- Yeni Hasta Verileri ile Tahmin ve Öneri Yapma ---
print("\n--- Analiz Sonuçları ---")
# Random Forest modellerini kullanarak tahmin yapalım
# predict_and_recommend fonksiyonuna feature_columns listesini geçirin
rf_results = predict_and_recommend(new_patient_df, rf_diagnosis_model, rf_stage_model, X.columns.tolist())
print("\n--- Random Forest Modelleri ile Sonuçlar ---")
for key, value in rf_results.items():
    print(f"{key}: {value}")

# Lojistik Regresyon modellerini kullanarak tahmin yapalım
# predict_and_recommend fonksiyonuna feature_columns listesini geçirin
lr_results = predict_and_recommend(new_patient_df, lr_diagnosis_model, lr_stage_model, X.columns.tolist())
print("\n--- Lojistik Regresyon Modelleri ile Sonuçlar ---")
for key, value in lr_results.items():
    print(f"{key}: {value}")

print("\n--- Önemli Not ---")
print("Bu sistem, mevcut veri setindeki kalıpları kullanarak bir ön teşhis ve öneri sunar.")
print("Ancak, yapay zeka tarafından yapılan tüm tahminler **sadece bilgilendirme amaçlıdır** ve tıbbi tavsiye yerine geçmez.")
print("Herhangi bir tıbbi teşhis, tedavi veya karar için daima nitelikli bir sağlık profesyoneline danışmanız esastır.")
print("Bu modelin doğruluğu, kullanılan veri setinin kalitesi, boyutu ve özelliklerinin kapsamına bağlıdır.")

--- Kansere Yakalanma Durumu (Teşhis) Modeli Sonuçları ---

Random Forest Sınıflandırıcısı (Teşhis) Eğitiliyor ve Değerlendiriliyor...
Doğruluk (Accuracy): 0.69
Sınıflandırma Raporu:
               precision    recall  f1-score   support

    0:LIVING       0.72      0.86      0.79        64
  1:DECEASED       0.57      0.36      0.44        33

    accuracy                           0.69        97
   macro avg       0.65      0.61      0.62        97
weighted avg       0.67      0.69      0.67        97


Lojistik Regresyon (Teşhis) Eğitiliyor ve Değerlendiriliyor...
Doğruluk (Accuracy): 0.65
Sınıflandırma Raporu:
               precision    recall  f1-score   support

    0:LIVING       0.71      0.78      0.75        64
  1:DECEASED       0.48      0.39      0.43        33

    accuracy                           0.65        97
   macro avg       0.60      0.59      0.59        97
weighted avg       0.64      0.65      0.64        97


--- Kanser Evresi Modeli Sonuçları ---

Random F

In [ ]:
X_train_diag, X_test_diag, y_train_diag, y_test_diag = train_test_split(X, y_diagnosis, test_size=0.2, random_state=42)
X_train_stage, X_test_stage, y_train_stage, y_test_stage = train_test_split(X, y_stage, test_size=0.2, random_state=42)

In [ ]:
# --- Modellerin Güvenilirliğini Test Etmek İçin Çapraz Doğrulama ---
print("\n--- Modellerin Güvenilirliği İçin Çapraz Doğrulama Sonuçları ---")

from sklearn.model_selection import KFold, cross_val_score # KFold ve cross_val_score import edildi

# K-Fold Çapraz Doğrulama için KFold nesnesi oluştur (5 kat)
cv_kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Kansere Yakalanma Durumu (Teşhis) Modelleri İçin Çapraz Doğrulama
print("\nKansere Yakalanma Durumu (Teşhis) Modelleri:")
# Teşhis modeli için çapraz doğrulama yapılıyor
rf_diagnosis_scores = cross_val_score(rf_diagnosis_model, X, y_diagnosis, cv=cv_kfold, scoring='accuracy')
print(f"Random Forest (Teşhis) Çapraz Doğrulama Doğrulukları: {rf_diagnosis_scores}")
print(f"Random Forest (Teşhis) Ortalama Doğruluk: {rf_diagnosis_scores.mean():.2f} (Std: {rf_diagnosis_scores.std():.2f})")

# Lojistik Regresyon (Teşhis) Modeli İçin Çapraz Doğrulama
lr_diagnosis_scores = cross_val_score(lr_diagnosis_model, X, y_diagnosis, cv=cv_kfold, scoring='accuracy')
print(f"Lojistik Regresyon (Teşhis) Çapraz Doğrulama Doğrulukları: {lr_diagnosis_scores}")
print(f"Lojistik Regresyon (Teşhis) Ortalama Doğruluk: {lr_diagnosis_scores.mean():.2f} (Std: {lr_diagnosis_scores.std():.2f})")


# Kanser Evresi Modelleri İçin Çapraz Doğrulama
print("\nKanser Evresi Modelleri:")
# Random Forest (Evre) Modeli İçin Çapraz Doğrulama
# Not: Daha önceki kodda evre modeli için yapılan çapraz doğrulamalar yanlışlıkla tekrar yazılmıştı, düzeltildi.
rf_stage_scores = cross_val_score(rf_stage_model, X, y_stage, cv=cv_kfold, scoring='accuracy')
print(f"Random Forest (Evre) Çapraz Doğrulama Doğrulukları: {rf_stage_scores}")
print(f"Random Forest (Evre) Ortalama Doğruluk: {rf_stage_scores.mean():.2f} (Std: {rf_stage_scores.std():.2f})")

# Lojistik Regresyon (Evre) Modeli İçin Çapraz Doğrulama
lr_stage_scores = cross_val_score(lr_stage_model, X, y_stage, cv=cv_kfold, scoring='accuracy')
print(f"Lojistik Regresyon (Evre) Çapraz Doğrulama Doğrulukları: {lr_stage_scores}")
print(f"Lojistik Regresyon (Evre) Ortalama Doğruluk: {lr_stage_scores.mean():.2f} (Std: {lr_stage_scores.std():.2f})")


--- Modellerin Güvenilirliği İçin Çapraz Doğrulama Sonuçları ---

Kansere Yakalanma Durumu (Teşhis) Modelleri:
Random Forest (Teşhis) Çapraz Doğrulama Doğrulukları: [0.70103093 0.67708333 0.60416667 0.67708333 0.58333333]
Random Forest (Teşhis) Ortalama Doğruluk: 0.65 (Std: 0.05)
Lojistik Regresyon (Teşhis) Çapraz Doğrulama Doğrulukları: [0.64948454 0.73958333 0.67708333 0.66666667 0.60416667]
Lojistik Regresyon (Teşhis) Ortalama Doğruluk: 0.67 (Std: 0.04)

Kanser Evresi Modelleri:
Random Forest (Evre) Çapraz Doğrulama Doğrulukları: [0.65979381 0.60416667 0.54166667 0.53125    0.58333333]
Random Forest (Evre) Ortalama Doğruluk: 0.58 (Std: 0.05)
Lojistik Regresyon (Evre) Çapraz Doğrulama Doğrulukları: [0.70103093 0.63541667 0.60416667 0.59375    0.65625   ]
Lojistik Regresyon (Evre) Ortalama Doğruluk: 0.64 (Std: 0.04)


In [ ]:
from sklearn.model_selection import KFold, cross_val_score